# **Tanda Tangan Digital (Keaslian & Anti-Sangkal)**
***Skenario Proyek: "Bank" mengirim "Laporan Keuangan" ke "Nasabah"***
# **`PERSIAPAN AWAL`**


**Kenapa perlu?** supaya modul kriptografi tersedia untuk membuat kunci, menandatangani, dan memverifikasi.

**Langkah:**

1. Pastikan Python 3.8+ terpasang.

2. Instal library:

In [ ]:
pip install cryptography


**Penjelasan:**

Kita menyiapkan Python dan memasang cryptography agar bisa membuat kunci RSA, menghitung hash, menandatangani data, dan memverifikasi tanda tangan.

# **Langkah 1: Bank Membuat Kunci (Identitas Digital)**
**Tujuan:** Setiap "Bank" harus memiliki sepasang kunci terlebih dahulu.


*   Private Key 🔐 (RAHASIA): Hanya dimiliki oleh Bank. Digunakan untuk membuat tanda tangan.
*   Public Key 🔑 (PUBLIK): Boleh disebar ke siapa saja (Nasabah/Publik). Digunakan untuk memeriksa tanda tangan.

**Kenapa perlu:** Digital signature hanya bisa dibuat oleh pemilik **private key**, dan diverifikasi bisa dilakukan oleh siapa pun yang punya **public key**.


In [ ]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa

# 1) Buat kunci RSA 3072-bit (aman & umum)
private_key = rsa.generate_private_key(public_exponent=65537, key_size=3072)
public_key = private_key.public_key()

# 2) Simpan private key ke file PEM (tanpa password dulu, biar simpel)
with open("bank_private.pem", "wb") as f:
    f.write(
        private_key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.PKCS8,
            encryption_algorithm=serialization.NoEncryption()  # bisa diganti password
        )
    )

# 3) Simpan public key ke file PEM
with open("bank_public.pem", "wb") as f:
    f.write(
        public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        )
    )

print("✅ Kunci dibuat: bank_private.pem & bank_public.pem")


✅ Kunci dibuat: bank_private.pem & bank_public.pem


**Penjelasan:**

Bank membuat sepasang kunci RSA (Rahasia & Public).

*   Private key disimpan sebagai bank_private.pem (rahasia)
*   public key disimpan sebagai bank_public.pem (boleh dibagikan ke Nasabah/Publik)


# **Langkah 2 — Bank menulis laporan (laporan.txt)**
**Tujuan:** membuat file yang akan ditandatangani.

**Kenapa perlu:** Tanda tangan selalu melekat ke data spesifik. Jika data berubah 1 byte pun, verifikasi akan gagal.

In [ ]:
content = """LAPORAN KEUANGAN BANK - Q4
Total Aset   : 1.250.000.000
Total Kewajiban : 830.000.000
Laba Bersih  : 120.000.000
Tanggal      : 01-11-2025
"""

with open("laporan.txt", "w", encoding="utf-8") as f:
    f.write(content)

print("✅ Laporan dibuat: laporan.txt")


✅ Laporan dibuat: laporan.txt


**Penjelasan:**

Bank menyiapkan laporan.txt berisi data keuangan yang akan dikirim ke Nasabah.

# **Langkah 3 — Bank menandatangani (membuat Digital Signature)**
**Tujuan:** menghasilkan file **laporan.sig** sebagai tanda tangan digital, atas isi **laporan.txt**.

**Kenapa perlu:** penerima nanti bisa memastikan:

* berkas benar-benar dibuat Bank (autentik).
* isi tidak berubah (integritas).

**Catatan teknis singkat:** Di RSA modern, yang “ditandatangani” bukan “hash yang dienkripsi” secara manual, melainkan pemanggilan API ***private_key.sign()*** yang secara internal melakukan hashing (SHA-256) + padding (RSA-PSS). Ini adalah praktik yang benar dan aman.


In [ ]:
import base64
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import serialization

# 1) Muat private key Bank
with open("bank_private.pem", "rb") as f:
    private_key = serialization.load_pem_private_key(f.read(), password=None)

# 2) Baca isi laporan
with open("laporan.txt", "rb") as f:
    data = f.read()

# 3) Buat tanda tangan
signature = private_key.sign(
    data,
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    hashes.SHA256()
)

# 4) Simpan tanda tangan ke file .sig (Base64)
with open("laporan.sig", "wb") as f:
    f.write(base64.b64encode(signature))

print("✅ Tanda tangan dibuat: laporan.sig (Base64)")


✅ Tanda tangan dibuat: laporan.sig (Base64)


**Penjelasan:**

Bank memanggil ***private_key.sign()*** dengan skema RSA-PSS & SHA-256 terhadap isi laporan.txt. Hasilnya disimpan sebagai laporan.sig (dipaketkan Base64 agar aman/nyaman saat kirim lewat email/HTTP).

# **Langkah 4 — Pengiriman ke Nasabah**
**Tujuan:** mengirim dua file: laporan.txt dan laporan.sig.

**Kenapa perlu:**


*   laporan.txt + laporan.sig harus dikirim bersamaan agar penerima bisa memverifikasi.
*   Public key Bank (bank_public.pem) harus sudah dimiliki atau diunduh oleh Nasabah dari sumber resmi Bank (misalnya website Bank atau QR code keamanan).

💡 Simulasi Sederhana

Di proyek Python lokal, kamu bisa anggap bahwa:

*   Folder “Bank” = sisi pengirim.
*   Folder “Nasabah” = sisi penerima.





🧾 Langkah-langkah Simulasi Pengiriman

1️⃣ Buat dua folder dulu:

In [ ]:
!mkdir Bank
!mkdir Nasabah


2️⃣ Pindahkan file yang dihasilkan oleh pihak Bank ke folder Bank:

In [ ]:
!mv bank_private.pem bank_public.pem laporan.txt laporan.sig Bank/


**Folder “Bank” berisi:**
*   bank_private.pem
*   bank_public.pem
*   laporan.txt
*   laporan.sig





3️⃣ Kirim ke Nasabah

In [ ]:
!cp Bank/laporan.txt Bank/laporan.sig Bank/bank_public.pem Nasabah/

| File               | Keterangan                                          | Dikirim ke Nasabah? |
| ------------------ | --------------------------------------------------- | ------------------- |
| `bank_private.pem` | Private key milik Bank (rahasia, **tidak dikirim**) | ❌                   |
| `bank_public.pem`  | Public key Bank (boleh dibagikan)                   | ✅                   |
| `laporan.txt`      | Isi laporan keuangan (dokumen utama)                | ✅                   |
| `laporan.sig`      | Tanda tangan digital atas dokumen                   | ✅                   |

Jadi tiga file yang boleh dikirim/dipindahkan adalah:

*   bank_public.pem
*   laporan.txt
*   laporan.sig





4️⃣ Verifikasi isi folder di sisi Nasabah

In [ ]:
%cd Nasabah
!python3 verify_signature.py
%cd ..


/content/Nasabah
✅ VERIFIKASI BERHASIL: Dokumen ASLI dari Bank dan tidak diubah.
/content


# **Langkah 5 — Nasabah memverifikasi tanda tangan**
**Tujuan:** memastikan bahwa

*   pengirim betul oleh Bank
*   isi dokumen tidak diubah.

**Kenapa perlu:** inilah inti autentikasi dan integritas; sekaligus dasar non-repudiation (Bank tidak bisa menyangkal karena hanya dia yang punya private key).



In [ ]:
%%writefile Nasabah/verify_signature.py
import base64, os, sys
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import serialization

def existing_path(*candidates):
    for p in candidates:
        if p and os.path.exists(p):
            return p
    return None

# Base dir eksekusi
CWD = os.getcwd()
NASABAH_DIR = os.path.join(CWD, "Nasabah")

# Kandidat lokasi file (cari di beberapa kemungkinan lokasi)
pub_candidates = [
    os.path.join(NASABAH_DIR, "bank_public.pem"),
    os.path.join(CWD, "bank_public.pem"),
    "bank_public.pem",
]
rep_candidates = [
    os.path.join(NASABAH_DIR, "laporan.txt"),
    os.path.join(CWD, "laporan.txt"),
    "laporan.txt",
]
sig_candidates = [
    os.path.join(NASABAH_DIR, "laporan.sig"),
    os.path.join(CWD, "laporan.sig"),
    "laporan.sig",
]

PUBLIC_KEY_PATH = existing_path(*pub_candidates)
REPORT_PATH     = existing_path(*rep_candidates)
SIGNATURE_PATH  = existing_path(*sig_candidates)

def die(msg):
    print("❌", msg)
    print("📂 CWD:", CWD)
    try:
        print("📄 Root contents:", os.listdir(CWD))
    except Exception:
        pass
    if os.path.exists(NASABAH_DIR):
        try:
            print("📄 Nasabah contents:", os.listdir(NASABAH_DIR))
        except Exception:
            pass
    sys.exit(1)

if not PUBLIC_KEY_PATH:
    die("bank_public.pem tidak ditemukan di lokasi yang umum.")
if not REPORT_PATH:
    die("laporan.txt tidak ditemukan di lokasi yang umum.")
if not SIGNATURE_PATH:
    die("laporan.sig tidak ditemukan di lokasi yang umum.")

# 1) Muat public key
with open(PUBLIC_KEY_PATH, "rb") as f:
    public_key = serialization.load_pem_public_key(f.read())

# 2) Baca laporan & tanda tangan
with open(REPORT_PATH, "rb") as f:
    data = f.read()
with open(SIGNATURE_PATH, "rb") as f:
    signature_b64 = f.read()
signature = base64.b64decode(signature_b64)

# 3) Verifikasi
try:
    public_key.verify(
        signature,
        data,
        padding.PSS(mgf=padding.MGF1(hashes.SHA256()),
                    salt_length=padding.PSS.MAX_LENGTH),
        hashes.SHA256()
    )
    print("✅ VERIFIKASI BERHASIL: Dokumen ASLI dari Bank dan tidak diubah.")
except Exception:
    die("VERIFIKASI GAGAL: Dokumen diubah atau tanda tangan tidak sah.")


Overwriting Nasabah/verify_signature.py


Kemudian jalankan:


In [ ]:
!python3 Nasabah/verify_signature.py


✅ VERIFIKASI BERHASIL: Dokumen ASLI dari Bank dan tidak diubah.
